In [73]:
import numpy as np 
import pandas as pd 
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


In [74]:
#Get the data
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
#create a copy to work on
df=data.copy()
print('data downloaded !')

data downloaded !


In [75]:
#let's create a clen dataframe
dx=df.loc[(df.Borough!='Not assigned'),:]

In [76]:
#rename 
dx.rename(columns={'Postal Code':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [77]:
dx.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [78]:
#size
dx.shape

(103, 3)

In [79]:
#get the postal code
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
print('data ready')

data ready


In [80]:
#let's process before merging
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
#merge our dataframe
dx=dx.merge(df_geo, on='PostalCode')

In [81]:
dx.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Clustering of our Dasaset 

In [82]:
# import k-means 
from sklearn.cluster import KMeans

### The purpose of our clustering will be to create zone according to the location of district 

#### Pre-processing
k-means algorithm isn't directly applicable to categorical variables because Euclidean distance function isn't really meaningful for discrete variables. So, lets drop this feature and run clustering.

In [83]:
#drop column
dx_cluster = dx[['Latitude','Longitude']]

In [84]:
dx_cluster.head()

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494


In [85]:
from sklearn.preprocessing import StandardScaler
X = dx_cluster
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

array([[ 0.93187104,  0.69818881],
       [ 0.40749844,  0.84388426],
       [-0.96437519,  0.37773518],
       [ 0.26644077, -0.6993678 ],
       [-0.81034434,  0.07922652],
       [-0.7039597 , -1.39737754],
       [ 1.95523876,  2.09777597],
       [ 0.79103364,  0.465121  ],
       [ 0.03427589,  0.90216906],
       [-0.90879146,  0.18842596],
       [ 0.09518258, -0.49568547],
       [-1.0279041 , -1.62993333],
       [ 1.53094905,  2.44798852],
       [ 0.40783172,  0.58164715],
       [-0.17744183,  0.81474393],
       [-1.01735586,  0.22482887],
       [-0.20737226, -0.32106485],
       [-1.1701819 , -1.86243118],
       [ 1.12942801,  2.15613628],
       [-0.54111407,  1.07704414],
       [-1.14613182,  0.24667041],
       [-0.29846415, -0.58298336],
       [ 1.271543  ,  1.86437197],
       [ 0.0852875 ,  0.3486083 ],
       [-0.89364811,  0.10106496],
       [-0.67165797, -0.26285143],
       [ 1.3126078 ,  1.6310228 ],
       [ 1.89922981,  0.3486083 ],
       [ 0.95236034,

In [86]:
clusterNum = 6
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[0 0 4 3 4 1 2 5 0 4 3 1 2 0 0 4 4 1 2 0 4 3 2 0 4 4 2 5 5 0 4 4 2 5 3 0 4
 4 2 5 3 0 4 4 0 5 3 0 4 3 3 2 5 3 0 5 3 3 0 5 3 5 4 3 3 2 5 4 4 3 3 2 5 4
 4 4 1 1 2 4 4 1 2 4 4 2 4 4 1 3 2 4 4 1 1 2 4 4 1 4 0 1 1]


In [87]:
dx["zone/cluster"] = labels
dx.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,zone/cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,0
1,M4A,North York,Victoria Village,43.725882,-79.315572,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4


In [88]:
import folium

print('Folium installed and imported!')

Folium installed and imported!


Torronto localisation 43.651070, -79.347015

In [89]:
# define the world map centered around Canada with a low zoom level
world_map = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

# display world map
world_map

In [90]:
k=clusterNum

In [91]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(clusterNum)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, Neighborhood, cluster in zip(dx['Latitude'], dx['Longitude'], dx['Neighborhood'], dx['zone/cluster']):
    label = folium.Popup(' Cluster '+ str(cluster) + ':'+ str(Neighborhood), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Observation

Torronto is cut in 6 areas according to the longitude and lattitude as we can see on the map 